In [3]:
# CSV FILE PATH
CSV_FILE_LOCATION = r"C:\Users\Kishan\Desktop\python_projects\csv_to_json\input\employee.csv"
SCHEMA_HEADER_COLUMNS = ""
SEPARATOR = ","
OUTPUT_LOCATION = r"C:\Users\Kishan\Desktop\python_projects\csv_to_json\output\employee.json"
LOG_FILE= r"C:\Users\Kishan\Desktop\python_projects\csv_to_json\logs\log_file.logs"
#CHECK FILE EXISTS

import os
print("CSV FILE LOCATION: {}".format(CSV_FILE_LOCATION))

CSV FILE LOCATION: C:\Users\Kishan\Desktop\python_projects\csv_to_json\input\employee.csv


In [35]:
import os
import pandas as pd
import logging
import sys

class csv_to_json:
    def __init__(self, input_file_path, output_file_path, column_names=None, delimiter=","):
        self.input_file_path = input_file_path
        self.output_file_path = output_file_path
        if(isinstance(column_names, list)):
            self.column_names = column_names
        elif(isinstance(column_names, str)):
            self.column_names = list(map(lambda x: x.strip(), column_names.split(",")))
        else:
            self.column_names = column_names
        self.delimiter = delimiter
        
    def file_exists_check(self):
        return os.path.exists(self.input_file_path)

    def is_file(self):
        return os.path.isfile(self.input_file_path)
    
    def transform_column_names(self, df):
        columns_dict = {}
        for i in map(lambda x: (x,x.strip().replace(' ','_').lower()), df.columns):
            columns_dict[i[0]] = i[1]
        return columns_dict

    def convert(self):
        if(self.column_names == None):
            df = pd.read_csv(self.input_file_path, delimiter=self.delimiter)
        else:
            df = pd.read_csv(self.input_file_path, delimiter=self.delimiter, names=self.column_names)
        df = df.rename(columns=self.transform_column_names(df))
        df.to_json(orient="records", path_or_buf=self.output_file_path)
        

In [46]:
def main():
    # Create a custom logger
    logger = logging.getLogger(__name__)
    # Create handlers
    c_handler = logging.StreamHandler()
    f_handler = logging.FileHandler('temp.log')
    c_handler.setLevel(logging.INFO)
    f_handler.setLevel(logging.ERROR)
    
    # Create formatters and add it to handlers
    c_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    f_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    c_handler.setFormatter(c_format)
    f_handler.setFormatter(f_format)

    # Add handlers to the logger
    logger.addHandler(c_handler)
    logger.addHandler(f_handler)
    
    converter = csv_to_json(CSV_FILE_LOCATION, OUTPUT_LOCATION)
    logger.info("PROCESS STARTED")
    if(converter.is_file()):
        logger.info("INPUT FILE PATH IS FILE '{}'".format(converter.input_file_path))
    else:
        logger.error("INPUT FILE PATH IS NOT FILE '{}'".format(converter.input_file_path))
        sys.exit(1)
        
    if(converter.file_exists_check()):
        logger.info("INPUT FILE EXISTS '{}'".format(converter.input_file_path))
    else:
        logger.error("INPUT FILE DOSEN'T EXISTS '{}'".format(converter.input_file_path))
        sys.exit(1)
    logger.info("CONVERTING CSV TO JSON")
    converter.convert()
    logger.info("FILE CONVERTED SUCCESSFULLY")
    logger.info("CHECK OUTPUT LOCATION: '{}'".format(converter.output_file_path))
    c_handler.close()
    f_handler.close()

In [47]:
main()

In [52]:
%%writefile ../scripts/csv_to_json.py
import argparse
import json
import logging
import os
import pandas as pd
import sys
from datetime import datetime
from json import JSONDecodeError


def read_config_file(file_path):
    config = {}
    try:
        with open(file_path, 'r') as conf:
            try:
                config = json.load(conf)
            except JSONDecodeError as e:
                logging.error("JSON Exception: {}".format(e))
                logging.error("EXITING")
                sys.exit(1)
    except FileNotFoundError as f:
        logging.error("File Not Found Exception: {}".format(f))
        logging.error("EXITING")
        sys.exit(1)
    return config


class csv_to_json:
    def __init__(self, input_file_path, output_file_path, column_names=None, delimiter=","):
        self.input_file_path = input_file_path
        self.output_file_path = output_file_path
        if (isinstance(column_names, list)):
            self.column_names = column_names
        elif (isinstance(column_names, str)):
            self.column_names = list(map(lambda x: x.strip(), column_names.split(",")))
        else:
            self.column_names = column_names
        self.delimiter = delimiter

    def file_exists_check(self):
        return os.path.exists(self.input_file_path)

    def is_file(self):
        return os.path.isfile(self.input_file_path)

    def transform_column_names(self, df):
        columns_dict = {}
        for i in map(lambda x: (x, x.strip().replace(' ', '_').lower()), df.columns):
            columns_dict[i[0]] = i[1]
        return columns_dict

    def convert(self):
        if (self.column_names == None):
            df = pd.read_csv(self.input_file_path, delimiter=self.delimiter, engine="python")
        else:
            df = pd.read_csv(self.input_file_path, delimiter=self.delimiter, names=self.column_names, engine="python")
        df = df.rename(columns=self.transform_column_names(df))
        df.to_json(orient="records", path_or_buf=self.output_file_path)


def main():
    parser = argparse.ArgumentParser(description='CSV to JSON Converter')

    parser.add_argument('-t', '--type', help='Configuration type JSON/ARGS', required=True)
    parser.add_argument('-c', '--confFilePath', help='JSON Configuration File path', default=None)
    parser.add_argument('-i', '--inputFilePath', help='Input CSV File Path', default=None)
    parser.add_argument('-o', '--outputFilePath', help='Output path to store JSON File', default=None)
    parser.add_argument('-d', '--delimiter', help='CSV File delimiter', default=",", type=str)
    parser.add_argument('-l', '--logFilePath', help='Log File Path', default=None)
    parser.add_argument('-n', '--columnNames', help='List of Column Names - For files without header', default=None)

    args = parser.parse_args()
    conf_type = args.type
    conf_file_path = args.confFilePath
    input_csv_file_path = args.inputFilePath
    ouput_json_file_path = args.outputFilePath
    delimiter = args.delimiter
    log_file_path = args.logFilePath
    column_names = args.columnNames

    if (conf_type.lower() == "json" and conf_file_path != None):
        config = read_config_file(conf_file_path)
        config_keys = config.keys()
        delimiter = ","
        column_names = None
        if "input_file" in config_keys:
            input_csv_file_path = config["input_file"]
        else:
            logging.error("input_file paramater not definied in the config JSON file")
            logging.error("EXITING")
            sys.exit()
        if "output_file" in config_keys:
            ouput_json_file_path = config["output_file"]
        else:
            logging.error("output_file paramater not definied in the config JSON file")
            logging.error("EXITING")
            sys.exit()
        if "delimiter" in config_keys:
            delimiter = config["delimiter"]
        if "column_names" in config_keys:
            column_names = config["column_names"]
        if "log_file_path" in config_keys:
            log_file_path = config["log_file_path"]
    elif (conf_type.lower() == "json" and conf_file_path == None):
        logging.error("--confFilePath is required arguments when type argument value is JSON")
        logging.error("EXITING")
        sys.exit(1)
    else:
        if (input_csv_file_path == None or ouput_json_file_path == None):
            logging.error(
                "--inputFilePath and --outputFilePath are required arguments when type argument value is not JSON")
            logging.error("EXITING")
            sys.exit(1)
    
    # Create a custom logger
    logger = logging.getLogger(__name__)
    current_timestamp = datetime.now().strftime("%Y-%m-%d_%H_%M_%S")
    log_file_name = 'csv_to_json_' + str(current_timestamp) + '.logs'
    if log_file_path != None:
        log_file_name = os.path.join(log_file_path, log_file_name)
    f_handler = logging.FileHandler(log_file_name)
    f_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    f_handler.setFormatter(f_format)
    f_handler.setLevel(logging.DEBUG)
    logger.addHandler(f_handler)
    
    logger.info("CONFIGURATION TYPE: {}".format(conf_type))
    if conf_file_path != None:
        logger.info("CONF FILE PATH: '{}'".format(conf_file_path))
    if input_csv_file_path != None:
        logger.info("INPUT CSV FILE PATH: '{}'".format(input_csv_file_path))
    if ouput_json_file_path != None:
        logger.info("OUTPUT JSON FILE PATH: '{}'".format(ouput_json_file_path))
    logger.info("DELIMITER: {}".format(delimiter))
    if log_file_path != None:
        logger.info("LOG FILE PATH: '{}'".format(log_file_path))
    if column_names != None:
        logger.info("LIST OF COLUMN NAMES: {}".format(column_names))

    logger.info("PROCESS STARTED")
    if column_names != None:
        converter = csv_to_json(input_csv_file_path, ouput_json_file_path, delimiter=delimiter, column_names=column_names)
    else:
        converter = csv_to_json(input_csv_file_path, ouput_json_file_path, delimiter=delimiter)

    if (converter.is_file()):
        logger.info("INPUT FILE PATH IS FILE '{}'".format(converter.input_file_path))
    else:
        logger.error("INPUT FILE PATH IS NOT FILE '{}'".format(converter.input_file_path))
        logger.error("EXITING..")
        sys.exit(1)

    if (converter.file_exists_check()):
        logger.info("INPUT FILE EXISTS '{}'".format(converter.input_file_path))
    else:
        logger.error("INPUT FILE DOSEN'T EXISTS '{}'".format(converter.input_file_path))
        logger.error("EXITING..")
        sys.exit(1)

    logger.info("CONVERTING CSV TO JSON")
    converter.convert()
    logger.info("FILE CONVERTED SUCCESSFULLY")
    logger.info("CHECK OUTPUT LOCATION: '{}'".format(converter.output_file_path))


if __name__ == "__main__":
    log_frmt = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    logging.basicConfig(level=logging.DEBUG, format=log_frmt)
    main()


Overwriting ../scripts/csv_to_json.py


In [45]:
!python ../scripts/csv_to_json.py -t ARGS -i ../input/employee.csv -o ../output/emp.json -n "empid, nameprefix, firstname, middleinitial, lastname, gender, email, father'sname, mother'sname, mother'smaidenname, dateofbirth, timeofbirth, ageinyrs., weightinkgs., dateofjoining, quarterofjoining, halfofjoining, yearofjoining, monthofjoining, monthnameofjoining, shortmonth, dayofjoining, dowofjoining, shortdow, ageincompany(years), salary, last%hike, ssn, phoneno., placename, county, city, state, zip, region, username, password" 

2020-05-24 18:01:55,821 - __main__ - INFO - CONFIGURATION TYPE: ARGS
2020-05-24 18:01:55,823 - __main__ - INFO - INPUT CSV FILE PATH: '../input/employee.csv'
2020-05-24 18:01:55,823 - __main__ - INFO - OUTPUT JSON FILE PATH: '../output/emp.json'
2020-05-24 18:01:55,823 - __main__ - INFO - DELIMITER: ,
2020-05-24 18:01:55,823 - __main__ - INFO - LIST OF COLUMN NAMES: empid, nameprefix, firstname, middleinitial, lastname, gender, email, father'sname, mother'sname, mother'smaidenname, dateofbirth, timeofbirth, ageinyrs., weightinkgs., dateofjoining, quarterofjoining, halfofjoining, yearofjoining, monthofjoining, monthnameofjoining, shortmonth, dayofjoining, dowofjoining, shortdow, ageincompany(years), salary, last%hike, ssn, phoneno., placename, county, city, state, zip, region, username, password
2020-05-24 18:01:55,824 - __main__ - INFO - PROCESS STARTED
2020-05-24 18:01:55,824 - __main__ - INFO - INPUT FILE PATH IS FILE '../input/employee.csv'
2020-05-24 18:01:55,824 - __main__ - INFO 

In [25]:
!python ../scripts/csv_to_json1.py -t json -c ../conf_files/conf.json

2020-05-24 17:49:53,046 - __main__ - INFO - CONFIGURATION TYPE: json
2020-05-24 17:49:53,047 - __main__ - INFO - CONF FILE PATH: '../conf_files/conf.json'
2020-05-24 17:49:53,047 - __main__ - INFO - INPUT CSV FILE PATH: 'C:\Users\Kishan\Desktop\python_projects\csv_to_json\input\employee.csv'
2020-05-24 17:49:53,047 - __main__ - INFO - OUTPUT JSON FILE PATH: 'C:\Users\Kishan\Desktop\python_projects\csv_to_json\output\employee.json'
2020-05-24 17:49:53,047 - __main__ - INFO - DELIMITER: ,
2020-05-24 17:49:53,047 - __main__ - INFO - LOG FILE PATH: 'C:\Users\Kishan\Desktop\python_projects\csv_to_json\logs'
2020-05-24 17:49:53,049 - __main__ - INFO - PROCESS STARTED
2020-05-24 17:49:53,049 - __main__ - INFO - INPUT FILE PATH IS FILE 'C:\Users\Kishan\Desktop\python_projects\csv_to_json\input\employee.csv'
2020-05-24 17:49:53,049 - __main__ - INFO - INPUT FILE EXISTS 'C:\Users\Kishan\Desktop\python_projects\csv_to_json\input\employee.csv'
2020-05-24 17:49:53,049 - __main__ - INFO - CONVERTING

In [26]:
!cat ../conf_files/conf.json

{
	"input_file" : "C:\\Users\\Kishan\\Desktop\\python_projects\\csv_to_json\\input\\employee.csv",
	"output_file" : "C:\\Users\\Kishan\\Desktop\\python_projects\\csv_to_json\\output\\employee.json",
	"delimiter" : ",",
	"log_file_path" : "C:\\Users\\Kishan\\Desktop\\python_projects\\csv_to_json\\logs"
}


In [218]:
config = read_config_file(conf_file_path)
config_keys = config.keys()
delimiter = ","
column_names = None
if "input_file" in config_keys:
    input_csv_file_path = config["input_file"]
else:
    print("input_file paramater not definied in the config JSON file")
    print("EXITING")
    sys.exit()
if "output_file" in config_keys:
    ouput_json_file_path = config["output_file"]
else:
    print("output_file paramater not definied in the config JSON file")
    print("EXITING")
    sys.exit()
if "delimiter" in config_keys:
    delimiter = config["delimiter"]
if "column_names" in config_keys:
    column_names = config["column_names"]
if "log_file_path" in config_keys:
    log_file_path = config["log_file_path"]

In [241]:
import argparse

In [199]:
print(input_csv_file_path)

C:\Users\Kishan\Desktop\python_projects\csv_to_json\input\employee.csv


In [39]:
p = r"C:\Users\Kishan\Desktop\python_projects\csv_to_json\input\employee.csv"

import pandas as pd

df = pd.read_csv(p)

In [43]:
", ".join(map(lambda x: x.lower().strip().replace(" ", ""), df.columns))

"empid, nameprefix, firstname, middleinitial, lastname, gender, email, father'sname, mother'sname, mother'smaidenname, dateofbirth, timeofbirth, ageinyrs., weightinkgs., dateofjoining, quarterofjoining, halfofjoining, yearofjoining, monthofjoining, monthnameofjoining, shortmonth, dayofjoining, dowofjoining, shortdow, ageincompany(years), salary, last%hike, ssn, phoneno., placename, county, city, state, zip, region, username, password"